# HDFS のデータ操作
BDC は、ストレージプールのレイヤーに HDFS のデータストアを標準で持っている。  
この HDFS 上のデータについては、T-SQL を使用して参照することができる。

## 1. Big Data Cluster の HDFS のデータにアクセス

In [6]:
USE [StoragePool];

-- オブジェクトの初期化
IF EXISTS (SELECT * FROM sys.external_tables WHERE name = 'StoragePoolTBL')
BEGIN
	DROP EXTERNAL TABLE StoragePoolTBL
END;
IF EXISTS (SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
	DROP EXTERNAL FILE FORMAT csv_file
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0536094

**1. HDFS 上のファイルのデータフォーマットを指定**  
外部データがどのようなフォーマットなのかを判断するための、ファイルフォーマットを作成

In [7]:
USE [StoragePool];

-- CSV の外部データフォーマットを作成
CREATE EXTERNAL FILE FORMAT csv_file
WITH (
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS(
        FIELD_TERMINATOR = ',',
        STRING_DELIMITER = '"',
        FIRST_ROW = 2,
        USE_TYPE_DEFAULT = TRUE)
);

Commands completed successfully.

Total execution time: 00:00:00.8107450

**2. 外部テーブルの作成**  
HDFS のデータを参照するための外部テーブルの作成  
Big Data Cluster 上の HDFS の指定したディレクトリ （今回は [/clickstream_Data]) をテーブルとして参照

In [8]:
USE [StoragePool];

-- Storage Pool の HDFS の内容を外部テーブルとして作成
CREATE EXTERNAL TABLE [StoragePoolTBL]
(
	wcs_click_date_sk BIGINT , 
	wcs_click_time_sk BIGINT , 
	wcs_sales_sk BIGINT , 
	wcs_item_sk BIGINT , 
	wcs_web_page_sk BIGINT , 
	wcs_user_sk BIGINT
)
WITH
(
    DATA_SOURCE = SqlStoragePool,
	LOCATION = '/clickstream_data',
    FILE_FORMAT = csv_file
);

Commands completed successfully.

Total execution time: 00:00:00.4115197

![BDC HDFS Data](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/03.Data%20Store/01.Storage%20Pool/BDC%20HDFS%20Data.png)

**3. HDFS 上のデータを参照**  
HDFS のデータをテーブルとして参照  
以降の操作はデータ仮想化と同様になる

In [9]:
USE [StoragePool];

-- Storage Pool の HDFS の内容を検索
SELECT COUNT(*) FROM [StoragePoolTBL]
SELECT TOP 20 * FROM [StoragePoolTBL] WHERE wcs_click_time_sk > 5000

(1 row affected)

(20 rows affected)

Total execution time: 00:00:04.4687193

(No column name)
1497


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
38569,85106,NULL,11130,18,NULL
38569,52655,NULL,3716,18,NULL
38569,70934,NULL,13243,18,NULL
38569,40166,NULL,5389,18,NULL
38570,73271,NULL,3331,18,NULL
38570,24651,NULL,10049,18,NULL
38570,23805,NULL,921,18,NULL
38570,66458,NULL,4407,18,NULL
38570,65912,NULL,11494,18,NULL
38570,80073,NULL,1833,18,NULL


![HDFS Data Query](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/03.Data%20Store/01.Storage%20Pool/Storage%20Pool%20Data%20Query.png)